
# Barbero durmiente

Para tratar el problema se ha implementado una clase que simula la Barbería con los procesos que se ejecutan dentro de ella (actividadBarbero y actividadCliente)
En el constructor de la Barberia se inicializan todos los cerrojos, semaforos, condiciones y variables que se necesitan para la implementación.
En el main se genera un hilo para la actividad del barbero, y hasta 20 hilos para la actividad del cliente, generados en un intervalo de tiempo aleatorio.

## Inicialización de variables
Se definen una serie de variables relativas a la concurrencia; exclusión mutua, de espera y de sincronización.

### Mecanismos y variables de control de clientes y acceso a la sala.

1. "MAX_CLIENTES": Entero con el número de clientes máximo que pueden estar esperando en la sala de espera. Con esta variable, se puede modificar el número de clientes máximo de manera dinámica de manera sencilla.
2. "entrarEnSala": Un semáforo limitado a 3 unidades, para controlar la entrada en la peluquería.
La idea es que al tratar de cruzar la puerta de la peluquería, el semáforo pone al cliente en espera en caso de que ya se hayan consumido 3 unidades (que equivalen a 3 personas en la sala de espera)
Se usa el parametro "timeout", para que en caso de que el cliente se quede bloqueado en la puerta, espere hasta un máximo de tiempo (establecido en 10 segundos para el ejemplo.)
Además, se mide el tiempo de espera de cada cliente en la puerta, para ilustrar con el paso de los clientes, si el barbero va bien de tiempo o si se le va acumulando el trabajo. 
3. "clientesEnSala": Entero que controla en tiempo real el número de clientes que hay en la sala de espera. Aunque el número máximo de clientes posibles en la sala de espera se controla con el semáforo "entrarEnSala", esta variable es útil para saber cuando no hay nadie en la sala y por tanto el Barbero se puede ir a dormir.
4. "exclusionMutuaSala": Un cerrojo para modificar las variables compartidas "clientesEnSala" y "barberoOcupado". La idea es poder modificar estas variables desde uno de los procesos asegurando que desde el otro no se están tratando de modificar a la vez, o incluso, asegurando que la consulta que se haga de ellas sea consistente.

### Mecanismos y variables de control del barbero.
1. "sillon": Condición que se usa para controlar el acceso al sillón de corte. La idea es que cuando un cliente esté en la sala de espera, solicite el cerrojo sillon, para a continuación quedarse en espera con el wait() de la condición (liberando el cerrojo sillon inmediatamente). Mientras que el Barbero, cuando comprueba que tiene clientes en sala de espera, cambia su estado a ocupado, y a continuación, tras obtener el cerrojo sillon, lanza una notificación para desbloquear el wait() del cliente. De esta manera, los clientes que van llegando a sala de espera, solicitan por orden el acceso al sillón, y acceden a él cada vez que el Barbero termina con el anterior Cliente. Se genera por tanto una cola FIFO.
2. "barberoOcupado": Variable booleana que indica cuando el barbero está ocupado (desde que comprueba que hay clientes en la sala de espera, hasta que termina de limpiar el sillón de corte); o que está libre (desde que termina de limpiar el sillón de corte hasta que comprueba si hay clientes en la sala de espera)
3. "dormir": Condición que se usa para controlar cuando el barbero debe ponerse a dormir o debe despertar. La idea es que el barbero se pone a dormir cuando no hay clientes en la sala. Notar que cuando el barbero duerme, previamente siempre ha estado en estado No ocupado. Así pues, cuando llega un cliente a la peluquería, comprueba si el barbero está libre, lo que quiere decir que tiene que despertarlo, y así lo hace. El mecanismo de esta condición es similar al de sillón. Se usan cerrojos en la propia variable para evitar problemas de consulta o escritura a la vez en los dos procesos, y posteriormente se usa el wait() y el notify() para dormri y despertar respectivamente.

## Actividad del barbero
En primer lugar se comprueba el número de clientes en la sala de espera con exclusión mutua "exclusionMutuaSala". Si no hay nadie, el barbero se va a dormir. Notar que antes del wait() de dormir, se libera la exclusión mutua para no bloquear la ejecución. Cuando se despierta, se vuelve a usar la exclusión mutua con el fin de modificar la variable "barberoOcupado".
En este punto, haya estado durmiendo o no, el barbero ya está en modo trabajo "barberoOcupado = True". Lo siguiente es avisar a los clientes que tiene el sillón de corte libre mediante un notify() de sillon. 
El tiempo estimado para que se siente el cliente, se corte el pelo y se limpie el sillón es de 2.5 segundos.
Finalmente se cambia el estado del barbero a "barberoOcupado = False" para indicar a los clientes que puedan estar en la sala que está disponible.
El bucle vuelve a repetirse de manera indefinida.

## Actividad del cliente
En primer lugar, el cliente llega a la puerta de la barbería. Por medio del semáforo "entrarEnSala", espera en la puerta hasta que haya sitio en la sala de espera. Si no hay sitio, permanece en la puerta esperando hasta un máximo de 10 segundos. En cuanto se libera un sitio, el cliente puede entrar. En ese caso, se mide el tiempo que le ha costado entrar, útil para hacer análisis de los procesos. Si pasan 10 segundos y no se ha liberado la sala de espera, el cliente simplemente se marcha (se usa return para terminar el hilo).
En caso de que haya conseguido entrar a la peluquería, comprueba si el barbero está ocupado, en cuyo caso se sienta a esperar (y se incrementa el valor de clientesEnSala); o si está disponible, en cuyo caso lo despierta (ya que estará durmiendo).
Sea como sea, lo que sigue es solicitar el sillón de corte por medio de un wait de la condición sillon.
Una vez se recibe la notificación por parte del barbero de que el sillon está libre, se sienta y libera un hueco del semáforo "entrarEnSala" para que un posible cliente que esté esperando en la puerta pueda entrar. Además de reducir en una unidad la variable clientesEnSala (con exclusión mutua, como siempre que se trabaja con esta variable compartida). Aún ocupando el sillón como cerrojo, el cliente tarda 1.5 segundos en que le corten el pelo. En cuanto pasa ese tiempo, simplemente termina el hilo, liberando el cerrojo del sillón, dejando paso al barbero para volver a avisar a otro cliente de que lo tiene libre.


In [1]:
import threading
import time
import random

class Barberia:
    def __init__(self):
        ### Mecanismos y variables de control de clientes y acceso a la sala. ###
        # Limita el tamaño máximo de clientes en la sala de espera.
        self.MAX_CLIENTES = 3 
        # Semáforo: Controla la entrada en la sala.
        self.entrarEnSala = threading.Semaphore(self.MAX_CLIENTES) 
        # Controla el número de clientes en la sala de espera.
        self.clientesEnSala = 0  
        # Cerrojo: Garantiza exclusion para modificacion de variables de control de sala o acceso a secciones
        self.exclusionMutuaSala = threading.Lock() 

        ### Mecanismos y variables de contrrol del barbero. ###
        # Condición: Controla el acesso al sillon de corte, puede poner clientes en espera y ser despertados
        self.sillon = threading.Condition()
        # Controla el estado del barbero.
        self.barberoOcupado = False
        # Condición: Puede poner al barbero a dormir y ser despertado
        self.dormir = threading.Condition()

    def actividadBarbero(self):
        while True:
            
            print ("\nBarbero: Clientes en la sala de espera: " + str(self.clientesEnSala))
            
            ### Se consulta en exclusion mutua los clientes que hay en la sala. ###
            with self.exclusionMutuaSala:
                if self.clientesEnSala == 0:
                    ### Si no hay clientes esperando se va a dormir hasta que le despierten. ####
                    print ("Barbero: No hay clientes. Se va a dormir.")
                    with self.dormir:    
                        # Se libera el lock de exclusión mutua de la sala para evutar un bloqueo. Se retoma cuando se despierta.
                        self.exclusionMutuaSala.release()
                        self.dormir.wait()
                        self.exclusionMutuaSala.acquire()

                ###  Si hay clientes, el barbero pasa a estar ocupado. ###
                self.barberoOcupado = True
                print("Barbero: Pasa a estar ocupado, listo para cortar el pelo.")
            
            ### Avisa de que tiene el sillón libre. ###
            with self.sillon:
                self.sillon.notify()
                print("Barbero: Tiene el sillón libre. Esperando que se siente un cliente.")

            time.sleep(2.5) # Tiempo que necesita el barbero para cortar el pelo y limpiar la zona de pelado.
            
            ###  El barbero se libera, dando pie a los clientes a que puedan  ###
            with self.exclusionMutuaSala:
                self.barberoOcupado = False
                print("Barbero: Ya ha pelado. Vuelve a estar libre.")
                    
            
    def actividadCliente(self):

        print(threading.current_thread().name + ": Llega a la puerta de la barbería. Espera en la puerta hasta que haya sitio en la sala de espera.")
        
        ### Según estado de la sala, entrar o esperar ####
        tiempo_inicial = time.time()
        if not self.entrarEnSala.acquire(timeout=10):  # Espera hasta 10 segundos para entrar en la sala.
            print(f"{threading.current_thread().name} : La sala de espera está ocupada demasado tiempo. Se va tras esperar 10 segundos.")
            return
        tiempo_final = time.time()
        print(threading.current_thread().name + ": Cruza la puerta. Ha esperado fuera durante " + str(tiempo_final - tiempo_inicial) + " segundos.")

        ### Consultar en exclusion mutua el estado ddel barbero. Cliente a sala de espera o a pelarse directamente. ###
        with self.exclusionMutuaSala:
            if self.barberoOcupado:
                self.clientesEnSala += 1
                print( threading.current_thread().name + ": Se sienta en la sala de espera: (" + str(self.clientesEnSala) + ")" )
            else:
                with self.dormir:
                    self.dormir.notify()
                    print (threading.current_thread().name + ": Despierta al barbero.")

        ### Cuando el sillón queda libre, se sienta a pelarse y libera un sitio en la sala de espera ###
        with self.sillon:
            self.sillon.wait()
            self.entrarEnSala.release()  
            with self.exclusionMutuaSala:
                if self.clientesEnSala > 0:
                    self.clientesEnSala -= 1 
        
            print (threading.current_thread().name + ": Se sienta en el sillón para pelarse. Libera un sitio de la sala de espera:(" + str(self.clientesEnSala) + ")")
                
            time.sleep(1.5) # Tiempo que dura un corte de pelo.

            ### El cliente se va pelado y deja el sillón libre. ###
            print (threading.current_thread().name + ": Libera el sillón. Abandona la barbería pelado.")

if __name__=="__main__":

        barberia = Barberia()
        
        # Se crea e inicia el hilo del barbero
        barbero = threading.Thread(target = barberia.actividadBarbero)
        barbero.start()
        
        # Se crea e inician los hilos para varios clientes
        for i in range(20):
            cliente = threading.Thread(target = barberia.actividadCliente, name = f'Cliente {i+1}')
            time.sleep(random.uniform(0.5, 2))  # Simula la llegada de clientes en momentos aleatorios
            cliente.start()




Barbero: Clientes en la sala de espera: 0
Barbero: No hay clientes. Se va a dormir.
Cliente 1: Llega a la puerta de la barbería. Espera en la puerta hasta que haya sitio en la sala de espera.
Cliente 1: Cruza la puerta. Ha esperado fuera durante 0.0 segundos.
Cliente 1: Despierta al barbero.
Barbero: Pasa a estar ocupado, listo para cortar el pelo.
Barbero: Tiene el sillón libre. Esperando que se siente un cliente.
Cliente 1: Se sienta en el sillón para pelarse. Libera un sitio de la sala de espera:(0)
Cliente 1: Libera el sillón. Abandona la barbería pelado.
Cliente 2: Llega a la puerta de la barbería. Espera en la puerta hasta que haya sitio en la sala de espera.
Cliente 2: Cruza la puerta. Ha esperado fuera durante 0.0 segundos.
Cliente 2: Se sienta en la sala de espera: (1)
Barbero: Ya ha pelado. Vuelve a estar libre.

Barbero: Clientes en la sala de espera: 1
Barbero: Pasa a estar ocupado, listo para cortar el pelo.
Barbero: Tiene el sillón libre. Esperando que se siente un clien

Cliente 10: Libera el sillón. Abandona la barbería pelado.
Barbero: Ya ha pelado. Vuelve a estar libre.

Barbero: Clientes en la sala de espera: 3
Barbero: Pasa a estar ocupado, listo para cortar el pelo.
Barbero: Tiene el sillón libre. Esperando que se siente un cliente.
Cliente 11: Se sienta en el sillón para pelarse. Libera un sitio de la sala de espera:(2)
Cliente 14: Cruza la puerta. Ha esperado fuera durante 9.49852204322815 segundos.
Cliente 14: Se sienta en la sala de espera: (3)
Cliente 11: Libera el sillón. Abandona la barbería pelado.
Cliente 15 : La sala de espera está ocupada demasado tiempo. Se va tras esperar 10 segundos.
Barbero: Ya ha pelado. Vuelve a estar libre.

Barbero: Clientes en la sala de espera: 3
Barbero: Pasa a estar ocupado, listo para cortar el pelo.
Barbero: Tiene el sillón libre. Esperando que se siente un cliente.
Cliente 12: Se sienta en el sillón para pelarse. Libera un sitio de la sala de espera:(2)
Cliente 16: Cruza la puerta. Ha esperado fuera dura

## Análisis de la ejecución
Para una correcta comprensión de la ejecución de los procesos concurrentes, se han desarrollado una serie de señalizaciones que permiten leer paso a paso qué ha pasado. 

En el ejemplo, para el trabajo del barbero (avisar del asiento disponible, pelar y limpiar el sillón) se establecen 2.5 segundos, mientras que para el corte de pelo del cliente se establecen 1.5 segundos. La llegada de clientes a la peluquería, por otra parte, se define de manera aleatoria entre 0.5 y 2 segundos. Con estos datos, la ejecución muestra como da tiempo a pelar a gran parte de los clientes, aunque siempre hay 3 o 4 que se van tras esperar más de 10 segundos. Si se modifican estos tiempos se pueden ver diferentes comportamientos en los procesos, siempre coherentes con la lógica buscada. Es decir:
- Si los clientes llegan a la peluquería con un lapso de tiempo más grande, al barbero le dará tiempo a cortar el pelo a todo el mundo y nadie se tendrá que ir tras esperar 10 segundos. O dicho de otra manera, el tiempo de corte del barbero tiene una relación indirecta con los clientes que puede antender, mientras que el tiempo de llegada de los clientes tiene una relación directa con los clientes a atender.
- El tiempo que hay de diferencia entre el tiempo de corte del barbero, y el tiempo de corte del cliente no es relevante para la eficiencia de la atención de los clientes. Aunque el tiempo de corte del cliente sea mucho más pequeño, el cuello de botella siempre será el tiempo de corte del barbero. Notar en cualquier caso, que no tiene sentido establecer el tiempo de corte del barbero más pequeño que el tiempo de corte del cliente. Este siempre debe ser igual o mayor. 